In [1]:
#FULL CODE:
#Importing libraries
import sys
import os
import re
import time

import json
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
import spacy
from spacy.matcher import PhraseMatcher, Matcher
from spacy.util import filter_spans
from spacy import displacy
from spacy.tokens import DocBin
from spacy.tokens import Span

tqdm.pandas()
#using pretrained gensim model for creating vocabulary
import gensim.downloader as api
import gensim


In [2]:
import gensim
from gensim.models import KeyedVectors
from nltk.tokenize import word_tokenize
import numpy as np
import copy

In [3]:
#Loading different embeddings function by choice #files located locally
def load_embed(embed):
    if embed == "fasttext":
        return  (KeyedVectors.load('saved_ft_embed',mmap='r'))
    elif embed == "glove":
        return  (KeyedVectors.load('saved_glove_embed',mmap='r'))
    elif embed == "conceptnet":
        #return (KeyedVectors.load(r'C:\Users\new\Documents\PythonFiles\conceptnet_embeddings\saved_numb_batch_embed',mmap='r'))
        return (KeyedVectors.load(r"C:\Users\DELL\Desktop\NLP files\Ner model files\conceptnet_embeddings\saved_numb_batch_embed",mmap='r'))
    else:   
        return None


In [4]:

# This function loads and preprocesses the corpus and the vocabulary
def LoadAndPreprocessInputs(corpus_filename, vocab_filename = None, keyword_list = None):
    print("preprocess functions is runnning")
    # Readin the corpus file
    corpus_data = open(corpus_filename, encoding='utf-8').read().strip()

    # split into patents texts | 1 entry = 1 patent
    #corpus_texts = corpus_data.split('\n\n')

    # split each patent into lines
    corpus_lines = corpus_data.split('\n')
    print(len(corpus_lines))
#     print(len(corpus_texts))
#     print(type(corpus_texts))
    #create our own vocublary using key words
        #-------------Need to implement---------------
        
    if vocab_filename.endswith('json'):
        f = open(vocab_filename)
        vocab = json.load(f)
        new_vocab = {}

        return corpus_lines, vocab#vocab_processed


In [5]:
def chunk_comparision(LABEL_NAME, line, nlp, embed_model, keyword_embedding, matcher_dict, keyword, line_index, updated_vocabulary):

    doc = nlp(line)
    chunk_dict = {}
    counter = 0
    for chunk in doc.noun_chunks:

        if str(chunk.root.text) in embed_model.key_to_index:
            similarity = get_cosine_similarity(keyword_embedding, embed_model[str(chunk.root.text)])
            if similarity > 0.6:
                # print("\tLine index: ", line_index)
                # print("\tLine: ",line)
                # print("\t\t keyword: ",keyword,"   chunk: ", chunk," chunk root word: ",str(chunk.root.text)," similarity: ",similarity)
                #pattern = nlp.make_doc(str(chunk))
                matcher_dict[LABEL_NAME].append(str(chunk))
                updated_vocabulary[LABEL_NAME].append(str(chunk))
                counter+=1
    return matcher_dict, counter, updated_vocabulary

In [6]:
def string_comparision(LABEL_NAME, line, nlp, keyword, matcher_dict, line_index):
    
    match =re.search("\W*("+keyword+")\W*",line)
    
    if match != None:
        # print("\t\tLine index: ", line_index)
        # print("\t\tLine: ",line)
        # print("\t\t\t keyword: ",keyword, '--- match :', match.group())
        #pattern = nlp.make_doc(str(match.group()))
        matcher_dict[LABEL_NAME].append(str(match.group()))
    return matcher_dict

In [7]:
def matching_with_embed(vocab_processed, text_lines, embed_model, nlp):
    matcher_dict = {}   
    # f_time = datetime.datetime.now()  #for saving file
    # file_time = f_time.strftime("%m-%d-%Y-%H-%M-%S")
    # file_name = "matcher_trial_"+file_time+".json"
    counter1 = 0
    updated_vocabulary = copy.deepcopy(vocab_processed)
    for LABEL_NAME in vocab_processed: #iterating through vocab
        # start_time = datetime.datetime.now()
        # matcher_dict["start_time"]=start_time.strftime("%m-%d-%Y, %H:%M:%S")
        print("Currently processing keyword :", LABEL_NAME)
        matcher_dict[LABEL_NAME] = []
        expanded_keywords = vocab_processed[LABEL_NAME]
        
        for keyword in list(expanded_keywords)[0:50]:
            _keyword = keyword.replace(' ','_')
            #print("the expanded word currently running is :", word)
            
#             print("\t------------------------------------")
#             print("\tWordnet keywords: ",keyword)
#            print(len(text_lines))
            for line_index, line in enumerate(text_lines): #iterating through corpus lines
                counter1=counter1+1
                if(counter1%1000 == 0):
                    print(f'Process is running -- current iteration is...{counter1}', end='\r')
                    #print("Process is running -- current iteration is...", counter1)
                
                #keyword_embedding=[]#np.zeros(300,)
                if _keyword in embed_model.key_to_index: #if only present in vocab 
                    #print('_keyword checking -->',_keyword)
                    keyword_embedding = embed_model[_keyword]
                    matcher_dict, counter, updated_vocabulary = chunk_comparision(LABEL_NAME, line, nlp, embed_model, keyword_embedding, matcher_dict, keyword, line_index, updated_vocabulary)
                    if counter == 0:
                        matcher_dict = string_comparision(LABEL_NAME, line, nlp, keyword, matcher_dict, line_index)
                else:
                    mather_dict = string_comparision(LABEL_NAME, line, nlp, keyword, matcher_dict, line_index)
                
            
            
        matcher_dict[LABEL_NAME] = list(set(matcher_dict[LABEL_NAME]))
        # end_time = datetime.datetime.now()
        # matcher_dict["end_time"]=end_time.strftime("%m-%d-%Y, %H:%M:%S")
         
        # f =open(file_name,'a')        #Saving file for checking the vocab
        # f.write(json.dumps(list(set(matcher_dict[LABEL_NAME])))+"\n")
        # f.close()
        print("Keyword completed: ",LABEL_NAME)
        
    return matcher_dict, updated_vocabulary

In [8]:
# This function processess Matcher and also splits train and test data.
def MatcherForGeneratingTrainData(vocab_processed, corpus_lines, embed_model):
    print("matcher func is running")
    try:
        nlp = spacy.load("en_core_web_lg")
    except:
        !python -m spacy download en_core_web_lg
        nlp = spacy.load("en_core_web_lg")
    matcher_dict, updated_vocabulary = matching_with_embed(vocab_processed, corpus_lines, embed_model, nlp)
    return matcher_dict, updated_vocabulary
    matcher = PhraseMatcher(nlp.vocab, attr="LOWER")   #initializing the matcher
    for keyword, patterns in matcher_dict.items():
        matcher.add(keyword, patterns)
      
    #splitting of corpus into train and test is done:
    train_lines, test_lines = train_test_split(corpus_lines, test_size=0.3, random_state=42)
    return matcher, train_lines, test_lines, nlp

In [9]:
def CustomNerModelMainFunc(corpus_filename, vocab_filename, testing_input = None):
    embed_model = load_embed('conceptnet')
    corpus_lines, vocab_processed = LoadAndPreprocessInputs(corpus_filename, vocab_filename)
    matcher_dict, updated_vocabulary = MatcherForGeneratingTrainData(vocab_processed, corpus_lines, embed_model)
    return matcher_dict, updated_vocabulary

In [10]:
#started at 12.29 p.m

In [11]:
%%time 
corpus_filename = r"C:\Users\DELL\Desktop\NLP files\Ner model files\CE_GF_corpus_small_40.txt"
vocab_filename =r"C:\Users\DELL\Desktop\NLP files\Ner model files\ESG_Keywords_Clean_4473.json"
matcher_dict, updated_vocabulary = CustomNerModelMainFunc(corpus_filename, vocab_filename) 

preprocess functions is runnning
12833
matcher func is running
Currently processing keyword : INVESTMENT


NameError: name 'get_cosine_similarity' is not defined

In [12]:
matcher_dict

NameError: name 'matcher_dict' is not defined

In [ ]:
# PREPARING DATASET


In [ ]:
vocab_filename = r"C:\Users\DELL\Desktop\NLP files\Ner model files\ESG_Keywords_Clean_4473.json"

In [ ]:
f = open(vocab_filename)
vocab = json.load(f)

In [ ]:
vocab

In [ ]:
new_vocab = {}
for key,val in vocab.items():
    new_vocab[key]= list(val.keys())
    
new_vocab
    

In [ ]:
def chunk_comparision(LABEL_NAME, line, nlp, embed_model, keyword_embedding, matcher_dict, keyword, line_index):
    #print("chunk func is running")
    doc = nlp(line)
    chunk_dict = {}
    counter = 0
    for chunk in doc.noun_chunks:
        #print('chunk ---> ', chunk)
        if chunk.root.text in embed_model.key_to_index:
            similarity = get_cosine_similarity(keyword_embedding, embed_model[str(chunk.root.text)])
            if similarity > 0.6:
                print("\tLine index: ", line_index)
                print("\tLine: ",line)
                print("\t\t keyword: ",keyword,"   chunk: ", chunk," chunk root word: ",str(chunk.root.text)," similarity: ",similarity)
                pattern = nlp.make_doc(str(chunk))
                matcher_dict[LABEL_NAME].append(str(pattern))
                counter+=1
    return matcher_dict, counter

def string_comparision(LABEL_NAME, line, keyword, matcher_dict, line_index):
    #print('string comaprision func is running')
    
    match =re.search("\W*("+keyword+")\W*",line)
    
    if match != None:
        print("\t\tLine index: ", line_index)
        print("\t\tLine: ",line)
        print("\t\t\t keyword: ",keyword, '--- match :', match.group())
#         if len(word_tokenize(match.group())) == 1 and match.group() in word_tokenize(line):
#             #print(keyword, ' -----> ', match) 
#             matcher_dict[LABEL_NAME].append(str(match.group()))
#         if len(word_tokenize(match.group())) > 1:
            #print(keyword, ' -----> ', match) 
        matcher_dict[LABEL_NAME].append(str(match.group()))
    return matcher_dict
                   
def matching_with_embed(vocab_processed, text_lines, embed_model, nlp):
    matcher_dict = {}   
    f_time = datetime.datetime.now()  #for saving file
    file_time = f_time.strftime("%m-%d-%Y-%H-%M-%S")
    file_name = "matcher_trial_"+file_time+".json"
    counter1 = 0
    for LABEL_NAME in vocab_processed: #iterating through vocab
        start_time = datetime.datetime.now()
        matcher_dict["start_time"]=start_time.strftime("%m-%d-%Y, %H:%M:%S")
        print("Currently processing keyword :", LABEL_NAME)
        matcher_dict[LABEL_NAME] = []
        expanded_keywords = vocab_processed[LABEL_NAME]
        #print('expanded_words-->', expanded_keywords[0:50])
        for keyword in expanded_keywords[0:50]:
            keyword = keyword.replace(' ','_')
            #print("the expanded word currently running is :", word)
            counter1=counter1+1
            if(counter1%10 == 0):
                print("Process is running.....", counter1)
#             print("\t------------------------------------")
#             print("\tWordnet keywords: ",keyword)
#            print(len(text_lines))
            for line_index, line in enumerate(text_lines[0:2000]): #iterating through corpus lines
                
                #keyword_embedding=[]#np.zeros(300,)
                if keyword in embed_model.key_to_index: #if only present in vocab
                    keyword_embedding = embed_model[keyword]
                    matcher_dict, counter = chunk_comparision(LABEL_NAME, line, nlp, embed_model, keyword_embedding, matcher_dict, keyword, line_index)
                    if counter == 0:
                        matcher_dict = string_comparision(LABEL_NAME, line, keyword, matcher_dict, line_index)
                else:
                    mather_dict = string_comparision(LABEL_NAME, line, keyword, matcher_dict, line_index)
                
            
            
        matcher_dict[LABEL_NAME] = list(set(matcher_dict[LABEL_NAME]))
        end_time = datetime.datetime.now()
        matcher_dict["end_time"]=end_time.strftime("%m-%d-%Y, %H:%M:%S")
         
        f =open(file_name,'a')        #Saving file for checking the vocab
        f.write(json.dumps(list(set(matcher_dict[LABEL_NAME])))+"\n")
        f.close()
        print("Keyword completed: ",LABEL_NAME)
    return matcher_dict


In [ ]:
#Loading different embeddings function by choice #files located locally
def load_embed(embed):
    if embed == "fasttext":
        return  (KeyedVectors.load('saved_ft_embed',mmap='r'))
    elif embed == "glove":
        return  (KeyedVectors.load('saved_glove_embed',mmap='r'))
    elif embed == "conceptnet":
        return (KeyedVectors.load(r'C:\Users\new\Documents\PythonFiles\conceptnet_embeddings\saved_numb_batch_embed',mmap='r'))
    else:   
        return None

#__________________________________________________________________________________________

# This function loads and preprocesses the corpus and the vocabulary
def LoadAndPreprocessInputs(corpus_filename, vocab_filename = None, keyword_list = None):
    print("preprocess functions is runnning")
    # Readin the corpus file
    corpus_data = open(corpus_filename, encoding='utf-8').read().strip()

    # split into patents texts | 1 entry = 1 patent
    #corpus_texts = corpus_data.split('\n\n')

    # split each patent into lines
    corpus_lines = corpus_data.split('\n')
    print(len(corpus_lines))
#     print(len(corpus_texts))
#     print(type(corpus_texts))
    #create our own vocublary using key words
        #-------------Need to implement---------------
        
    if vocab_filename.endswith('json'):
        f = open(vocab_filename)
        vocab = json.load(f)
        new_vocab = {}
#         for key,val in vocab.items():
#             new_vocab[key]= list(val.keys())
        #print(vocab)
        #vocab_processed = VocabPreprocess(new_vocab, corpus_texts)
        #print(vocab_processed)
        #print(vocab_process)
        return corpus_lines, vocab#vocab_processed
    # Using existing vocablary 
    if vocab_filename != None:
        vocab = open(vocab_filename, encoding = 'utf-8').read().lower().strip().split('\n')
    elif keyword_list != None:
        vocab = CreatingVocabulary(keyword_list)
    #vocab_processed = VocabPreprocess(vocab, corpus_texts)
    return corpus_texts, corpus_lines, vocab

#__________________________________________________________________________________________



#__________________________________________________________________________________________

#Creating vocabulary through embeddings (conceptnet, fasttext,glove) when we dont have the terms.
def CreatingVocabulary(keyword_list):
    concept, fasttext, glove  = LoadEmbeddings()
    vocab_list = []
    embed_list = ["concept", "fasttext", "glove"]

    for embed in embed_list:
        each_embed_vocab_list=[]
        for keyword in keyword_list:
            each_key_vocab = []
            keyword_similar = embed.most_similar(keyword, topn = 5000)
            for word in keyword_similar:
                each_key_vocab.append(word[0].replace("_", " "))
                each_key_vocab.append(word[0].replace("-", " "))
            each_embed_vocab_list.append(each_key_vocab)
    vocab_list.append(each_embed_vocab_list)
    vocab_list = list(set(vocab_list))
    return vocab_list

#__________________________________________________________________________________________

# This function creates possible bigrams, trigrams for the vocabulary
def VocabPreprocess(vocab,corpus_texts):
    if type(vocab == dict):
        new_vocab = {}
        print(type(corpus_texts),'---',len(corpus_texts))
        for key in vocab:
            cvectorizer = CountVectorizer(ngram_range=(1, 4), stop_words="english", vocabulary=vocab[key], lowercase=True)
            X = cvectorizer.fit_transform(corpus_texts)
            #print(X)
            termdf_cv = pd.DataFrame(np.sum(X, axis=0), columns=cvectorizer.get_feature_names()).T.sort_values(by=0, ascending=False)
            new_vocab[key]=termdf_cv.index
            #print(new_vocab[key])
        return new_vocab  
    # Here lowercase=False option is used to keep the original case of the terms, since we possibly could have term abbreviations. Like API, CAT, etc.
    cvectorizer = CountVectorizer(ngram_range=(1, 4), stop_words="english", vocabulary=vocab, lowercase=True)
    X = cvectorizer.fit_transform(corpus_texts)

    # Show top-25 most frequent terms
    termdf_cv = pd.DataFrame(np.sum(X, axis=0), columns=cvectorizer.get_feature_names()).T.sort_values(by=0, ascending=False)
    #print(termdf_cv)
    return termdf_cv

from tqdm import tqdm
import datetime

def matching_with_embed(vocab_processed, text, embed_model, nlp):
    matcher_dict = {}   
    f_time = datetime.datetime.now()
    file_time = f_time.strftime("%m-%d-%Y-%H-%M-%S")
    file_name = "matcher_trial_"+file_time+".json"
    for key in tqdm(vocab_processed):
        start_time = datetime.datetime.now()
        matcher_dict["start_time"]=start_time.strftime("%m-%d-%Y, %H:%M:%S")
        print("Currently processing keyword :", key)
        matcher_dict[key] = []
        expanded = vocab_processed[key]
        for word in expanded:
            #print("the expanded word currently running is :", word)
            keyword_embedding=[]#np.zeros(300,)
            if word in embed_model.key_to_index:
                keyword_embedding = embed_model[word]
            
            for line in text:
                doc = nlp(line)
                chunk_list = {}
                for chunk in doc.noun_chunks:
                    chunk_list[str(chunk)]=str(chunk.root.text)
                if word in line:
                    pattern = nlp.make_doc(word)
#                     print(line)
#                     print(key, '---->1', pattern)
                    matcher_dict[key].append(str(pattern))
                elif len(keyword_embedding)>0:# != np.zeros(300,):
                    tokens = word_tokenize(line)
                    sent_embed = {}
                    for tok in tokens:
                        if tok in embed_model.key_to_index:
                            tok_embed = embed_model[tok]
                            sent_embed[tok]= tok_embed
                    for emb in  sent_embed:
                        score = get_cosine_similarity(keyword_embedding, sent_embed[emb])
                        if score > 0.7:
                            emb_list=[]
                            for chun in chunk_list:
                                if emb in chun:
                                    pattern = nlp.make_doc(str(chun))
#                                     print(line)
#                                     print(key, '---->2', pattern)
                                    matcher_dict[key].append(str(pattern))
                                elif(not emb in emb_list):
                                    pattern = nlp.make_doc(str(emb))
#                                     print(line)
#                                     print(key, '---->3', pattern)
                                    matcher_dict[key].append(str(pattern))
                                    emb_list.append(str(emb))
        end_time = datetime.datetime.now()
        matcher_dict["end_time"]=end_time.strftime("%m-%d-%Y, %H:%M:%S")
        
        f =open(file_name,'a')
        f.write(json.dumps(list(set(matcher_dict[key])))+"\n")
        f.close()
        print("Keyword completed: ",key)
    return matcher_dict
#__________________________________________________________________________________________
 # cosine similarity function
from sklearn.metrics.pairwise import cosine_similarity
def get_cosine_similarity(feature_vec_1, feature_vec_2):    
    return cosine_similarity(feature_vec_1.reshape(1, -1), feature_vec_2.reshape(1, -1))[0][0]



# This function processess Matcher and also splits train and test data.
def MatcherForGeneratingTrainData(vocab_processed, corpus_lines, embed_model, LABEL_NAME):
    print("matcher func is running")
    try:
        nlp = spacy.load("en_core_web_lg")
    except:
        !python -m spacy download en_core_web_lg
        nlp = spacy.load("en_core_web_lg")
    matcher_dict = matching_with_embed(vocab_processed, corpus_lines, embed_model, nlp)
    matcher = PhraseMatcher(nlp.vocab, attr="LOWER")   #initializing the matcher
    for key, patterns in matcher_dict.items():
        matcher.add(key, patterns)
      
    #splitting of corpus into train and test is done:
    train_lines, test_lines = train_test_split(corpus_lines, test_size=0.3, random_state=42)
    return matcher, train_lines, test_lines, nlp, matcher_dict

# # This function processess Matcher and also splits train and test data.
# def MatcherForGeneratingTrainData(vocab_processed, corpus_lines, LABEL_NAME):
#     print("matcher func is running")
#     try:
#         nlp = spacy.load("en_core_web_lg")
#     except:
#         !python -m spacy download en_core_web_lg
#         nlp = spacy.load("en_core_web_lg")
#     matcher = PhraseMatcher(nlp.vocab, attr="LOWER")   #initializing the matcher
#     if type(vocab_processed) == dict:
#       for key in vocab_processed:
#         #print("key-->",key, 'expanded--->', vocab_processed[key])
#         patterns = [nlp.make_doc(text) for text in list(vocab_processed[key])]  #tokenization is done and converted to a list of doc objects.
#         #print(vocab_processed)
#         matcher.add(key, patterns)
#     else:
#       patterns = [nlp.make_doc(text) for text in vocab_processed]  #tokenization is done and converted to a list of doc objects.
#       matcher.add(LABEL_NAME, patterns)
      
#     #splitting of corpus into train and test is done:
#     train_lines, test_lines = train_test_split(corpus_lines, test_size=0.3, random_state=42)
#     return matcher, train_lines, test_lines, nlp

#__________________________________________________________________________________________
def find_key(input_dict, value):
    for key, val in input_dict.items():
        if value in val: return key
    return None


# This func can create annotated dataset in .spacy format (recognizable by spacy)
def CreateSpacyAnnotatedDataset(text, n_lines, filename, matcher, LABEL_NAME, nlp, vocab_processed, offset=0):
    print("custom spacy dataset generation is running")
    LABEL = None
    doc_bin = DocBin()  # create a DocBin object #serialization object

    print("Dataset creation has begun ----->", filename)
    #n_lines = len(text)
    for training_example in tqdm(text[offset:offset+n_lines]):
        doc = nlp.make_doc(training_example)  #tokenization is done on each line and converted to a list
        ents = []
        #matches = matcher(doc)
        for match_id, start, end in matcher(doc):
            LABEL_NAME = find_key(vocab_processed, str(doc[start]))
            print("document--->",doc,"Label_name-->",LABEL_NAME)
            if LABEL_NAME is not None:
              span = Span(doc, start, end, label=LABEL_NAME)
              print("document--->",doc,"Label_name-->",LABEL_NAME)
              if span is None:
                  print("Skipping entity")
              else:
                  ents.append(span)

        filtered_ents = filter_spans(ents) #filters removes extra spans and duplicates
        #print(len(filtered_ents))
        #print(filtered_ents)
        doc.ents = filtered_ents
        doc_bin.add(doc)
    doc_bin.to_disk(filename)
    return filename

#__________________________________________________________________________________________

# This function initializes the config file and starts the training process
def CustomNerModelTraining(train_filename, test_filename):
    print("model_training is running")
    # Run to generate full training config
    !python -m spacy init fill-config /content/drive/MyDrive/Colab_Files/GFCE/base_config.cfg config.cfg
    
    #code for training the NER
    #code for training the NER
    !python -m spacy train config.cfg --output /content/drive/MyDrive/Colab_Files/GFCE/spacy_output --paths.train /content/drive/MyDrive/Colab_Files/GFCE/training_data.spacy --paths.dev /content/drive/MyDrive/Colab_Files/GFCE/training_data.spacy
    return "/content/drive/MyDrive/Colab_Files/GFCE/spacy_output/model-best"

#__________________________________________________________________________________________
    
# This function is necessary if you want to test your model on a text data.
def NerModelTesting(testing_input, model_path = "/spacy_output/model-best"):
    print("model_testing is running")
    #Testing the model
    nlp = spacy.load(model_path)

    doc = nlp(testing_input)

    colors = {"CRIME": "#F67DE3"}
    options = {"colors": colors}

    spacy.displacy.render(doc, style="ent", options=options, jupyter=True)

#__________________________________________________________________________________________

# This is the main function
def CustomNerModelMainFunc(corpus_filename, vocab_filename, LABEL_NAME, testing_input = None):
    embed_model = load_embed('conceptnet')
    corpus_texts, corpus_lines, vocab_processed = LoadAndPreprocessInputs(corpus_filename, vocab_filename)
    matcher, train_lines, test_lines, nlp, matcher_dict = MatcherForGeneratingTrainData(vocab_processed, corpus_lines[0:100], embed_model, LABEL_NAME)
#     train_filename = "/content/drive/MyDrive/Colab_Files/GFCE/training_data.spacy"
#     test_filename = "/content/drive/MyDrive/Colab_Files/GFCE/valid_data.spacy"

    # if not os.path.exists(train_filename):
    #train_filename = CreateSpacyAnnotatedDataset(train_lines, len(train_lines), train_filename, matcher, LABEL_NAME, nlp,vocab_processed)
    # if not os.path.exists(test_filename):
    #test_filename = CreateSpacyAnnotatedDataset(test_lines, len(test_lines), test_filename, matcher, LABEL_NAME, nlp, vocab_processed)
    #print('filenames---->',train_filename, test_filename)

    #model_path = CustomNerModelTraining(train_filename, test_filename)
#     if testing_input != None:
#         NerModelTesting(testing_input, model_path)
    return matcher_dict#model_path matcher 

In [ ]:
%%time
corpus_filename = r'C:\Users\new\Documents\PythonFiles\Hearst_pattern\ESG_train+valid_corpus_small_150.txt'
vocab_filename =r"C:\Users\new\Documents\PythonFiles\KOS algorithms and saved embeds\whole_dictionary.json"
#vocab_filename = ["rape", "robbery", "murder", "mutilation", "violence" ]
testing_input = "Wi-Fi Direct (registered trademark, which will be hereinafter referred to as WFD) \
               corresponding to a technology for directly performing a communication based on a \
               wireless LAN between communication devices without intermediation of an access \
               point (hereinafter referred to as AP) is standardized in Wi-Fi Alliance serving \
               as a wireless LAN industry group. My name is Kasi working at Leadsemantics"
LABEL_NAME = "INVESTMENT"

#Checking whether the files exists before the program
if os.path.exists(corpus_filename):
    print("corpus file exists")
else:
    print("File path no found")

if type(vocab_filename) == str:
    if os.path.exists(vocab_filename):
        print("vocab file exists")
    else:
        print("File path no found")


#Calling Function
model_path = CustomNerModelMainFunc(corpus_filename, vocab_filename, LABEL_NAME, testing_input) 

***

In [ ]:
df = pd.read_excel(r'C:\Users\new\Documents\PythonFiles\Hearst_pattern\final_vocab_analysis_2.xlsx')

In [ ]:
df.head()

In [ ]:
import json
f = open(r'C:\Users\new\Documents\PythonFiles\Hearst_pattern\fina_vocab_dict_600.json','r')
data = json.load(f)

In [ ]:
data

In [ ]:
label_list = []
extended_words_list = []
for enum, LABEL in enumerate(data):
    for enum1,word in enumerate(data[LABEL]):
        extended_words_list.append(word)
        label_list.append(LABEL)
    

In [ ]:
len(label_list)

In [ ]:
len(extended_words_list)

In [ ]:
percentile_list = pd.DataFrame(
    {'lst1Title': lst1,
     'lst2Title': lst2,
     'lst3Title': lst3
    })

In [ ]:
dataframe = pd.DataFrame({
    'ner': extended_words_list,
    'ner_type': label_list
})

In [ ]:
dataframe.tail()

In [ ]:
dataframe.to_csv('ESG_ner_and_ner_type_600.csv')

In [ ]:
import re
str1="get together"
text ="They are plannig for get togethers by sitting together."
word =re.search("\W*("+str1+")\W*",text)
print(word)


In [ ]:
#pip install --upgrade gensim